***Конвертация JSON2HTML***

Выполнил: Юрий Куськов y.kuskov@solarl.ru

---

In [20]:
from json import loads
from collections import OrderedDict

In [25]:
def read_json(file):
    try:
        f = open(file,'r')
    except IOError as e:
        print("Не удалось открыть файл")
    else:
        with f:
            st = f.read()
            f.close()
            return loads(st, object_pairs_hook=OrderedDict)

---

**1 задание**
Формат: список (параграф:заголовок)

In [3]:
def func_1(file):
    """в параметре file указывается путь к файлу 'folder/file.json'"""
    return "".join(["<h1>{}</h1><p>{}</p>".format(x["title"],x["body"]) for x in read_json(file)])

In [5]:
func_1('1st/source.json')

'<h1>Title#1</h1><p>Hello,World1!</p><h1>Title#2</h1><p>Hello,World2!</p>'

---

**2 задание**
В ключ добавлено название тега

In [6]:
def func_2(file):
    """в параметре file указывается путь к файлу 'folder/file.json'"""
    _in = read_json(file)
    _out = ""
    for x in _in:                     # цикл по словарям
        for k in x:                   # цикл внутри словаря
            _out += "<{0}>{1}</{0}>".format(k,x[k])
    return _out

In [7]:
func_2('2nd/source.json')

'<h3>Title#1</h3><div>Hello,World1!</div>'

---

**3 задание**
Если *list* - то все элементы, которые содержатся - должны быть обернуты в *ul*, а каждый конкретный элемент в списке в тег *li*

In [8]:
def func_3(file):
    """в параметре file указывается путь к файлу 'folder/file.json'"""
    _in = read_json(file)
    if isinstance(_in,list):
        _out = ""
        for x in _in:
            _out +="<li>"
            for k in x:
                _out += "<{0}>{1}</{0}>".format(k,x[k])
            _out +="</li>"
        #_out += "</ul>"
        return "<ul>"+_out+"</ul>"
    else: raise ValueError("Объект не является списком")

In [9]:
func_3('3rd/source.json') 

'<ul><li><h3>Title#1</h3><div>Hello,World1!</div></li><li><h3>Title#2</h3><div>Hello,World2!</div></li></ul>'

---

**4 задание**
Теперь список может появиться в любом месте, а элементы могут быть вложены друг в друга

In [10]:
def treeHTML(s):  
    """функция рекурсивного анализа json"""
    if isinstance(s,list):
        return "<ul>"+"".join(["<li>"+treeHTML(d)+"</li>" for d in s]) +"</ul>" 
    if isinstance(s,dict):
        _out = ""
        for k in s:
            if isinstance(s[k],list):
                _out += "<{0}>{1}</{0}>".format(k,treeHTML(s[k]))
            else:
                _out += "<{0}>{1}</{0}>".format(k,s[k])
        return _out

In [11]:
def func_4(file):
    """в параметре file указывается путь к файлу 'folder/file.json'"""
    _in = read_json(file)
    return treeHTML(_in)

In [12]:
func_4('4th/source.json')

'<ul><li><span>Title#1</span><content><ul><li><p>Example1</p><header>header1</header></li></ul></content></li><li><div>div1</div></li></ul>'

---

**5 задание**
Верстка поплыла - необхоимо добавлять класс и идентификаторы к тегам, а содержимое не должно рассматриваться как html

In [53]:
import re

def get_tag_classes_id(text: str) -> (str, str, str):
    """парсинг значения key из dict на (tag,classes,id)"""
    class_ = []
    id_ = ''

    tag, *items = re.split('[#.]', text)
    prefixs = re.findall('[#.]', text)

    for prefix, value in zip(prefixs, items):
        if prefix == '#':
            id_ = value
        else:
            class_.append(value)

    return tag, ' '.join(class_), id_

In [14]:
from html import escape   # замена символов на их соответсвующий html-код

In [54]:
def newTreeHTML(s):  
    """функция рекурсивного анализа json"""
    if isinstance(s,list):
        return "<ul>"+"".join(["<li>"+newTreeHTML(d)+"</li>" for d in s]) +"</ul>" 
    if isinstance(s,dict):
        _out = ""
        for k in s:
            _tag, _class, _id = get_tag_classes_id(k)
            if isinstance(s[k],list):
                if _class != '' and _id !="":
                    _out += '<{0} id="{2}" class="{1}">{3}</{0}>'.format(_tag, _class, _id, newTreeHTML(s[k]))
                elif _class != '' and _id == "":
                    _out += '<{0} class="{1}">{2}</{0}>'.format(_tag, _class, newTreeHTML(s[k]))
                elif _class == '' and _id != "":
                    _out += '<{0} id="{1}">{2}</{0}>'.format(_tag, _id, newTreeHTML(s[k]))
                else:
                    _out += '<{0}>{1}</{0}>'.format(_tag, newTreeHTML(s[k]))      
            else:
                if _class != '' and _id !="":
                    _out += '<{0} id="{2}" class="{1}">{3}</{0}>'.format(_tag, _class, _id, escape(s[k]))
                elif _class != '' and _id == "":
                    _out += '<{0} class="{1}">{2}</{0}>'.format(_tag, _class, escape(s[k]))
                elif _class == '' and _id != "":
                    _out += '<{0} id="{1}">{2}</{0}>'.format(_tag, _id, escape(s[k]))
                else:
                    _out += '<{0}>{1}</{0}>'.format(_tag, escape(s[k])) 
        return _out

In [55]:
def func_5n(file):
    """в параметре file указывается путь к файлу 'folder/file.json'"""
    _in = read_json(file)
    return newTreeHTML(_in)

In [60]:
func_5n('5th/source2.json') 

'<p id="my-id" class="my-class">hello</p><p class="my-class1 my-class2"><ul><li><div id="my-id3" class="my-clas2s">hello2</div></li></ul></p>'

---

# тест

In [17]:
import unittest
 
class TestNotebook(unittest.TestCase):
    def test_func_1(self):
        self.assertEqual(func_1('1st/source.json'), '<h1>Title#1</h1><p>Hello,World1!</p><h1>Title#2</h1><p>Hello,World2!</p>')
        
    def test_func_2(self):
        self.assertEqual(func_2('2nd/source.json'), '<h3>Title#1</h3><div>Hello,World1!</div>')
        
    def test_func_3(self):
        self.assertEqual(func_3('3rd/source.json') , '<ul><li><h3>Title#1</h3><div>Hello,World1!</div></li><li><h3>Title#2</h3><div>Hello,World2!</div></li></ul>')
        
    def test_func_4(self):
        self.assertEqual(func_4('4th/source.json'), '<ul><li><span>Title#1</span><content><ul><li><p>Example1</p><header>header1</header></li></ul></content></li><li><div>div1</div></li></ul>')
        
    def test_func_5(self):
        self.assertEqual(func_5('5th/source.json'), '<p id="my-id" class="my-class">hello</p><p class="my-class1 my-class2">example&lt;a&gt;asd&lt;/a&gt;</p>')
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_func_1 (__main__.TestNotebook) ... ok
test_func_2 (__main__.TestNotebook) ... ok
test_func_3 (__main__.TestNotebook) ... ok
test_func_4 (__main__.TestNotebook) ... ok
test_func_5 (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.033s

OK
